In [6]:
from random import randint

在做模拟之前，我们需要之前定义的两个类：SQueue和prioQueue

### SQueue的定义

In [136]:
# 队列类的实现
class SQueue():
    def __init__(self, init_len = 8):
        self._len = init_len # 存储区长度
        self._elems = [0] * init_len # 元素存储
        self._head = 0 # 表头元素下标
        self._num = 0 # 元素个数
    
    def is_empty(self):
        return self._num == 0

    def peek(self):
        if self._num == 0:
            raise QueueUnderflow
        return self._elems[self._head]
    
    def dequeue(self):
        if self._num == 0:
            raise QueueUnderflow
        e = self._elems[self._head]
        self._head = (self._head + 1)%self._len
        self._num -= 1
        return e
    
    def enqueue(self, e):
        if self._num == self._len:
            self.__extend()
        self._elems[(self._head + self._num) % self._len] = e
        self._num += 1
        
    def __extend(self):
        old_len = self._len
        self._len *= 2
        new_elems = [0]*self._len
        for i in range(old_len):
            new_elems[i] = self._elems[(self._head + i) % old_len]
            self._elems, self._head = new_elems, 0

In [182]:
a = SQueue()
a.enqueue(3)
a.enqueue(5)
a.dequeue()
a.peek()

5

### prioQueue的定义

In [146]:
class PrioQueue:
    """
    利用堆来实现优先队列
    """
    def __init__(self, elist = []):
        self._elems = list(elist) # 这一步是使内部的表脱离原来的表，防止元素的共享
        if elist:
            self.buildheap()
    
    def is_empty(self):
        return not self._elems
    
    def peek(self):
        if self.is_empty():
            raise PrioQueueError("in peek")
        return self._elems[0]
    
    # 入队操作
    def enqueue(self, e):
        self._elems.append(None) # add a dummy elemnt
        self.siftup(e, len(self._elems)-1)
        
    def siftup(self, e, last):
        # 初始化，(元素-1//2)对应的是其根节点
        elems, i, j = self._elems, last, (last-1)//2 
        # 判断，当e小于其根节点的时候，把根节点一段一段往下移动
        while i > 0 and e < elems[j]:
            elems[i] = elems[j]
            i, j = j, (j-1)//2
        elems[i] = e
    
    # 弹出操作
    def dequeue(self):
        if self.is_empty():
            raise PrioQueueError("in queue")
        elems = self._elems
        e0 = elems[0]
        e = elems.pop()
        if len(elems) > 0:
            self.siftdown(e, 0, len(elems))
        return e0
    
    def siftdown(self, e, begin, end):
        # 初始化，begin*2+1找的是其左根结点
        elems, i, j = self._elems, begin, begin * 2 + 1
        while j < end:
            # 判断右结点（兄弟结点）的情况
            if j+1 < end and elems[j+1] < elems[j]:
                j += 1 # 保证elems[j]里的数据不大于其兄弟结点的数据
            # 这样就保证了根节点一定会小于兄弟结点中较小的那个
            if e < elems[j]: 
                break # 因为e已经在三者中最小，找到了位置
            elems[i] = elems[j] # 因为元素j在三者中最小，所以将其上移。
            i, j = j, j * 2 + 1 # 更新i，j指针
        elems[i] = e
    
    # 堆的初始构建
    def buildheap(self):
        end = len(self._elems)
        # 从end//2开始做是因为，后面的元素已经是叶结点（顺序表转化为二叉树的特有性质）
        # 所以，它们中的每一个都已经是一个堆了；
        # 从完全二叉树的最下最右分支结点开始，往左建堆
        # 然后再到上一层建堆；
        # 最后，整个表成为一个堆
        # 这个工作实际上就是在两个已有的堆上加上一个根元素，使其成为堆
        for i in range(end//2, -1, -1):
            self.siftdown(self._elems[i], i, end)

Simulation是一个基础的模拟对象

In [147]:
class Simulation:
    def __init__(self, duration):
        self._eventq = PrioQueue() # 一个优先队列？
        self._time = 0
        self._duration = duration
        
    def run(self):
        while not self._eventq.is_empty(): # 模拟到事件队列空
            event = self._eventq.dequeue()
            self._time = event.time()
            if self._time > self._duration: # 同理，当时间用完时，也停止
                break
            event.run()
    
    def add_event(self, event):
        self._eventq.enqueue(event)
        
    def cur_time(self):
        return self._time    

接下来定义一个基础的事件类

In [148]:
class Event:
    def __init__(self, event_time, host):
        self._ctime = event_time
        self._host = host
    
    def __lt__(self, other_event):
        return self._ctime < other_event._ctime
    def __le__(self, other_event):
        return self._ctime <= other_event._ctime
    
    def host(self):
        return self._host
    
    def time(self):
        return self._ctime
    
    def run(self): # 具体事件类必须定义这个方法
        pass

### 模拟类：Customs
接下来就是针对这些需求，提出正确的拟合方法

模拟的前提如下：
* 车辆的速率具有一定的随机性，每a到b分钟有一辆车到达；
* 海关有k条检查通道，每检查一条一辆车要耗费c到d分钟
* 到达的车辆在一条专用线路上等待，一旦有一个检查通道空闲，正在等待的车辆就进入该通道检查（dequeue操作）
* 如果车辆到达有空闲通道而且当时没有等待车辆，就立刻开始检查

目的：得到车辆的平均等待时间和通过检查站的平均时间

In [171]:
class Customs:
    def __init__(self, gate_num, duration,
                arrive_interval, check_interval):
        self.simulation = Simulation(duration)
        self.waitline = SQueue()
        self.duration = duration
        self.gates = [0] * gate_num
        self.total_wait_time = 0
        self.total_used_time = 0
        self.car_num = 0
        self.arrive_interval = arrive_interval
        self.check_interval = check_interval
        
    def wait_time_acc(self, n):
        self.total_wait_time += n
    
    def total_time_acc(self, n):
        self.total_used_time += n
    
    def car_count_1(self):
        self.car_num += 1
    
    def add_event(self, event):
        self.simulation.add_event(event)
    
    def cur_time(self):
        return self.simulation.cur_time()
    
    def enqueue(self, car):
        self.waitline.enqueue(car)
    
    def has_queued_car(self):
        return not self.waitline.is_empty()
    
    def next_car(self):
        return self.waitline.dequeue()
    
    def find_gate(self):
        for i in range(len(self.gates)):
            if self.gates[i] == 0:
                self.gates[i] = 1
                return i
        return None
    
    def free_gate(self, i):
        if self.gates[i] == 1:
            self.gates[i] = 0
        else:
            raise ValueError("Clear gate error.")
            
    # 最后的方法实施模拟和输出统计数据
    def simulate(self):
        Arrive(0, self) # 初始化，生成第一辆汽车
        self.simulation.run()
        self.statistics()
        
    def statistics(self):
        print("Simulate " + str(self.duration)
                + " minutes, for "
                + str(len(self.gates)) + "gates.")
        print(self.car_num, "cars pass the Customs")
        print("Average waiting time:",
             self.total_wait_time/self.car_num)
        print("Average passing time:",
             self.total_used_time/self.car_num)
        i = 0
        while not self.waitline.is_empty():
            self.waitline.dequeue()
            i += 1
        print(i, "cars are in waiting line.")            

### Car类
排队队列列的对象表示同行的车辆，类Car实现这种对象，对象中记录了一辆车的到达时间。

In [170]:
class Car:
    def __init__(self, arrive_time):
        self.time = arrive_time
    
    def arrive_time(self):
        return self.time

### 事件类
海关检查的系统模拟需要以下几种事件：
* **汽车到达事件** ， 一辆车到达生成一个Car事件，其中记录了它的到达时间；同时，这一事件的发生还意味着若干时间之后还有下一辆车要达到；故而定义一个类Arrive，规范此类事件的行为；
* **汽车开始检查事件** 
* **汽车检查完毕后离开** ，定义类Leave

In [183]:
# 这个用来输出日志条目,在测试的时候就超级好用！
def event_log(time, name):
    print("Event: "+ name + ", happen at " + str(time))
    pass

In [152]:
# 首先是Arrive类
class Arrive(Event):
    def __init__(self, arrive_time, customs):
        Event.__init__(self, arrive_time, customs)
        customs.add_event(self)
        
    def run(self):
        time, customs = self.time(), self.host()
        event_log(time, "car arrive")
        # 生成下一个Arrive事件
        Arrive(time + randint(*customs.arrive_interval),
               customs)
        
        # 这一次到达车辆事件的行为
        car = Car(time)
        if customs.has_queued_car():
            customs.enqueue(car)
            return
        i = customs.find_gate() # 检查空闲通道
        if i is not None:
            event_log(time, "car check")
            Leave(time + randint(*customs.check_interval),
                 i, car, customs)
        else:
            customs.enqueue(car)

In [153]:
# 定义离开类
class Leave(Event):
    def __init__(self, leave_time, gate_num, car, customs):
        Event.__init__(self, leave_time, customs)
        self.car = car
        self.gate_num = gate_num
        customs.add_event(self)
        
    def run(self):
        time, customs = self.time(), self.host()
        event_log(time, "car leave")
        customs.free_gate(self.gate_num)
        customs.car_count_1()
        customs.total_time_acc(time - self.car.arrive_time())
        # 如果一辆车离开车站后，还有排队等待的车辆
        # 就检查其中第一辆车，并且生成相应的离开事件
        if customs.has_queued_car():
            car = customs.next_car()
            i = customs.find_gate()
            event_log(time,"car check")
            customs.wait_time_acc(time - car.arrive_time())
            Leave(time + randint(*customs.check_interval),
                 self.gate_num, car, customs)

## 实际模拟
虽然上面那一通都是我抄书的，但是还是可以尝试一下模拟


In [184]:
car_arrive_interval = (1,2)
car_check_time = (4, 5)
cus = Customs(3, 480, car_arrive_interval, car_check_time)
cus.simulate()

Event: car arrive, happen at 0
Event: car check, happen at 0
Event: car arrive, happen at 1
Event: car check, happen at 1
Event: car arrive, happen at 2
Event: car check, happen at 2
Event: car leave, happen at 4
Event: car arrive, happen at 4
Event: car check, happen at 4
Event: car leave, happen at 6
Event: car leave, happen at 6
Event: car arrive, happen at 6
Event: car check, happen at 6
Event: car arrive, happen at 7
Event: car check, happen at 7
Event: car leave, happen at 8
Event: car arrive, happen at 9
Event: car check, happen at 9
Event: car arrive, happen at 10
Event: car leave, happen at 11
Event: car check, happen at 11
Event: car arrive, happen at 11
Event: car leave, happen at 12
Event: car check, happen at 12
Event: car arrive, happen at 12
Event: car leave, happen at 13
Event: car check, happen at 13
Event: car arrive, happen at 14
Event: car leave, happen at 16
Event: car check, happen at 16
Event: car arrive, happen at 16
Event: car leave, happen at 16
Event: car che

Event: car arrive, happen at 436
Event: car leave, happen at 436
Event: car check, happen at 436
Event: car leave, happen at 437
Event: car check, happen at 437
Event: car arrive, happen at 437
Event: car arrive, happen at 438
Event: car leave, happen at 439
Event: car check, happen at 439
Event: car arrive, happen at 440
Event: car leave, happen at 441
Event: car check, happen at 441
Event: car leave, happen at 441
Event: car check, happen at 441
Event: car arrive, happen at 442
Event: car leave, happen at 443
Event: car check, happen at 443
Event: car arrive, happen at 443
Event: car arrive, happen at 444
Event: car arrive, happen at 445
Event: car leave, happen at 446
Event: car check, happen at 446
Event: car leave, happen at 446
Event: car check, happen at 446
Event: car arrive, happen at 447
Event: car leave, happen at 448
Event: car check, happen at 448
Event: car arrive, happen at 448
Event: car arrive, happen at 449
Event: car leave, happen at 450
Event: car check, happen at 4